## data

In [19]:
import pandas as pd

f_cost = pd.read_excel("D:/Uni/4/OR-1/project/OR1_Project_Data_.xlsx" ,sheet_name='f_cost')
mf_cost = pd.read_excel("D:/Uni/4/OR-1/project/OR1_Project_Data_.xlsx" ,sheet_name='mf_cost')
cf_cost = pd.read_excel("D:/Uni/4/OR-1/project/OR1_Project_Data_.xlsx" ,sheet_name='cf_cost')
return_request = pd.read_excel("D:/Uni/4/OR-1/project/OR1_Project_Data_.xlsx" ,sheet_name='return_request')

# convert into dictionary
Dict_f_cost = dict()
Dict_mf_cost = dict()
Dict_cf_cost = dict()
Dict_return_request = dict()
f_cost.set_index("a", inplace=True)
mf_cost.set_index("a", inplace=True)
cf_cost.set_index("a", inplace=True)
return_request.set_index("a", inplace=True)
for i in f_cost.index:
    Dict_f_cost [i] = round(float(f_cost['b'][i]),1)

for i in mf_cost.index:
    for j in mf_cost.columns:
        Dict_mf_cost[j,i] = round(float(mf_cost[j][i]),2)
        
for i in cf_cost.index:
    for j in cf_cost.columns:
        Dict_cf_cost[i,j] = round(float(cf_cost[j][i]),2)

for i in return_request.index:
    for j in return_request.columns:
        Dict_return_request[i,j] = round(float(return_request[j][i]) ) 

print ('Dict_f_cost:'  , Dict_f_cost )
print()
print ('Dict_mf_cost:' , Dict_mf_cost)
print()
print ('Dict_cf_cost:' , Dict_cf_cost)
print()
print ('Dict_return_request :' , Dict_return_request)

Dict_f_cost: {1: 1175.0, 2: 587.5, 3: 822.5, 4: 587.5, 5: 1175.0, 6: 1057.5, 7: 705.0, 8: 587.5}

Dict_mf_cost: {(1, 1): 2.11, (2, 1): 3.41, (3, 1): 3.71, (1, 2): 1.92, (2, 2): 4.01, (3, 2): 3.84, (1, 3): 1.04, (2, 3): 4.43, (3, 3): 4.51, (1, 4): 0.78, (2, 4): 2.7, (3, 4): 2.72, (1, 5): 1.54, (2, 5): 1.9, (3, 5): 1.93, (1, 6): 3.94, (2, 6): 1.14, (3, 6): 0.72, (1, 7): 3.64, (2, 7): 0.36, (3, 7): 0.25, (1, 8): 3.48, (2, 8): 0.5, (3, 8): 0.0}

Dict_cf_cost: {(1, 1): 8.62, (1, 2): 1.75, (1, 3): 6.09, (1, 4): 4.02, (1, 5): 4.47, (1, 6): 6.71, (1, 7): 7.17, (1, 8): 6.62, (2, 1): 4.74, (2, 2): 8.36, (2, 3): 7.88, (2, 4): 5.01, (2, 5): 3.69, (2, 6): 4.68, (2, 7): 3.13, (2, 8): 3.35, (3, 1): 5.09, (3, 2): 7.16, (3, 3): 7.3, (3, 4): 4.05, (3, 5): 2.55, (3, 6): 3.74, (3, 7): 2.42, (3, 8): 2.42, (4, 1): 2.56, (4, 2): 6.4, (4, 3): 2.44, (4, 4): 3.86, (4, 5): 4.88, (4, 6): 9.6, (4, 7): 8.63, (4, 8): 8.46, (5, 1): 2.41, (5, 2): 7.91, (5, 3): 3.7, (5, 4): 5.28, (5, 5): 6.12, (5, 6): 10.65, (5, 7): 9.

## modeling

In [20]:
from pulp import *


problem = LpProblem("minimize", LpMinimize)

x = LpVariable.dicts("x",[ (i,j) for i in range(1,21) for j in range(1,9) ] ,
                     cat='Binary',lowBound = 0)
y = LpVariable.dicts("y",[ (k,j) for k in range(1,4) for j in range(1,9) ] ,
                     cat='Integer' ,lowBound = 0)
w = LpVariable.dicts("w",[ (k,j) for k in range(1,4) for j in range(1,9) ] ,
                     cat='Integer' ,lowBound = 0)
K = LpVariable.dicts("K",[ j for j in range(1,9) ] ,
                     cat='Binary' ,lowBound = 0)
n = LpVariable.dicts("n",[ j for j in range(1,9) ] ,
                     cat='Integer' ,lowBound = 0)

# objective func
problem +=  lpSum(2*[(Dict_return_request[i,1] + Dict_return_request[i,2] + Dict_return_request[i,3]) 
                    * x[i,j] * Dict_cf_cost[i,j] for i in range(1,21) for j in range(1,9)]
                    + [ (w[k,j]+y[k,j]) * Dict_mf_cost[k,j] for k in range(1,4) for j in range(1,9) ]
                    + [ n[j] * Dict_f_cost[j] for j in range(1,9)]  ) 


# S.t

#1
problem += lpSum([ n[j] for j in range(1,9) ]) <= 269

for j in range(1,9):
#2
    problem +=  n[j] <= 130
#3
    problem += lpSum([(Dict_return_request[i,1] + Dict_return_request[i,2]
                       + Dict_return_request[i,3])*x[i,j] 
                      for i in range(1,21)]) <= 5 * lpSum( [y[k,j] for k in range(1,4)])
#4
    problem += lpSum([(Dict_return_request[i,1] + Dict_return_request[i,2]
                       + Dict_return_request[i,3])*x[i,j] 
                      for i in range(1,21)]) <= 3.3333 * lpSum( [w[k,j] for k in range(1,4)])
#5
    problem += lpSum([(8*Dict_return_request[i,1] + 10*Dict_return_request[i,2] 
                      + 12*Dict_return_request[i,3])*x[i,j] 
                     for i in range(1,21)]) <= 117.5 * n[j]  
#6
    problem += n[j] <= lpSum( 130 * [ x[i,j] for i in range(1,21)]) 

    
#8
    problem += lpSum([ (y[k,j] + w[k,j]) for k in range(1,4)]) == lpSum([
            ( Dict_return_request[i,1] + Dict_return_request[i,2]
            + Dict_return_request[i,3])*x[i,j] for i in range(1,21)] )
#9
    problem += n[j] <= 130 * K[j]
    problem += n[j] >= K[j]    

#7
for i in range(1,21):
    problem += lpSum([ x[i,j] for j in range(1,9) ]) == 1
    


problem.solve()


Optimal


## result

In [25]:
print('objective function value:' ,problem.objective.value())
for v in problem.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

objective function value: 181456.34
K_2 = 1.0
K_4 = 1.0
K_8 = 1.0
n_2 = 30.0
n_4 = 109.0
n_8 = 130.0
w_(1,_2) = 110.0
w_(1,_4) = 378.0
w_(3,_8) = 1208.0
x_(1,_2) = 1.0
x_(10,_2) = 1.0
x_(11,_8) = 1.0
x_(12,_4) = 1.0
x_(13,_8) = 1.0
x_(14,_4) = 1.0
x_(15,_2) = 1.0
x_(16,_8) = 1.0
x_(17,_8) = 1.0
x_(18,_4) = 1.0
x_(19,_8) = 1.0
x_(2,_4) = 1.0
x_(20,_8) = 1.0
x_(3,_8) = 1.0
x_(4,_4) = 1.0
x_(5,_4) = 1.0
x_(6,_8) = 1.0
x_(7,_4) = 1.0
x_(8,_4) = 1.0
x_(9,_8) = 1.0
y_(1,_2) = 254.0
y_(1,_4) = 880.0
y_(3,_8) = 303.0
